In [1]:
#导入所需库
#其中function_update是用于问卷数据聚合计算的函数和维度拆分函数等，具体注释参见function_update.py
import numpy as np
import pandas as pd
from function_update import *

In [2]:
#测试强制四舍五入
round(32.5)

33.0

In [3]:
#设置路径
import os
os.chdir('./data')

In [4]:
#导入数据，输出用于function表的index起始终止标记点,之后在function表上设置适用的函数和对应参数
#说明：数据已脱敏。数据label信息以label+序号代替、问卷数据的题目标题以question+序号代替
rawdata=pd.read_excel('input/sample_data_脱敏.xlsx')
rawdata_list = rawdata.columns.to_list()
make_index(rawdata_list)

In [7]:
#读取数据，这一步中列名会以序号代替，方便指定维度拆分的列
qc,d = readxl('sample_data_脱敏.xlsx',sheet_name='data')

In [8]:
qc.shape

(144, 273)

In [9]:
qc.head()

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,...,q264,q265,q266,q267,q268,q269,q270,q271,q272,q273
0,1,肿瘤内科,肿瘤内科-一线城市,NaN,NaN,NaN,一线城市,三级专科医院,肿瘤内科,副主任医师,...,是,否,否,NaN,"1. <10,000元人民币/月",2.现有BRAF抑制剂医保前价格,NaN,0.3100,0.269200,0.620
1,2,肿瘤内科,肿瘤内科-二线城市,NaN,NaN,NaN,二线城市,三级综合医院,肿瘤内科,主任医师,...,否,否,否,NaN,"2.10,000-20,000元人民币/月",4.患者经济能力,NaN,0.0925,0.239900,0.625
2,3,肿瘤外科,肿瘤外科-二线城市,NaN,NaN,NaN,二线城市,三级综合医院,胃肠外科,副主任医师,...,否,否,否,NaN,"1. <10,000元人民币/月",4.患者经济能力,NaN,0.0430,0.128409,0.215
3,4,肿瘤内科,肿瘤内科-二线城市,NaN,NaN,NaN,二线城市,三级专科医院,肿瘤内科,主任医师,...,否,否,否,NaN,"3.20,000-30,000元人民币/月",2.现有BRAF抑制剂医保前价格,NaN,0.0730,0.403654,0.685
4,5,肿瘤内科,肿瘤内科-二线城市,NaN,NaN,NaN,二线城市,三级综合医院,肿瘤内科,主任医师,...,否,否,否,NaN,"2.10,000-20,000元人民币/月",1.现有标准治疗方案的价格（如化疗联合贝伐珠单抗）,NaN,0.0390,0.171750,0.420


In [12]:
#使用break down函数，指定分类列序号，输出顺序可指定或默认，参见function_update.py,所有break down维度存到input_dict列表里
input_dict = {}
input_dict['ALL'] = qc
input_dict = break_down(qc,2,input_dict)
input_dict = break_down(qc,7,input_dict,['一线城市', '二线城市', '三线城市'])
input_dict = break_down(qc,3,input_dict,['肿瘤内科-一线城市', '肿瘤内科-二线城市', '肿瘤内科-三线城市', '肿瘤外科-一线城市', '肿瘤外科-二线城市', '肿瘤外科-三线城市'])

['肿瘤内科', '肿瘤外科']
['一线城市', '二线城市', '三线城市']
['肿瘤内科-一线城市', '肿瘤内科-二线城市', '肿瘤内科-三线城市', '肿瘤外科-一线城市', '肿瘤外科-二线城市', '肿瘤外科-三线城市']


In [13]:
#检查维度拆分是否正确
for k in input_dict:
    #print(k)
    print(k,input_dict[k].shape)

ALL (144, 273)
肿瘤内科 (108, 273)
肿瘤外科 (36, 273)
一线城市 (32, 273)
二线城市 (64, 273)
三线城市 (48, 273)
肿瘤内科-一线城市 (24, 273)
肿瘤内科-二线城市 (47, 273)
肿瘤内科-三线城市 (37, 273)
肿瘤外科-一线城市 (8, 273)
肿瘤外科-二线城市 (17, 273)
肿瘤外科-三线城市 (11, 273)


In [14]:
#导入函数参数表，函数参数表是在make_index后得到的问卷题目和起始终止标记点后，再设置具体使用的函数类型和对应的参数
#这里需要根据报告所需的数据人工设置
func_input = pd.read_excel('function/function_row_sample.xlsx')

In [15]:
func_input

,title,start,end,function,n,nan,vc_base,index,columns
0,Question7,7,7,vc,NaN,NaN,NaN,NaN,7
1,Question8,8,8,vc,NaN,NaN,NaN,NaN,8
2,Question9,9,9,vc,NaN,NaN,NaN,NaN,9
3,Question10,10,10,vc,NaN,NaN,NaN,NaN,10
4,Question11,11,11,vc,NaN,NaN,NaN,NaN,11
...,...,...,...,...,...,...,...,...,...
78,Question246,246,250,vc,NaN,NaN,shape,NaN,246 250
79,Question255,255,259,mean_percentage,NaN,NaN,NaN,NaN,255 259
80,Question261,261,266,vc,NaN,NaN,NaN,NaN,261 266
81,Question268,268,268,vc,NaN,NaN,NaN,NaN,268


In [16]:
#将function表转为字典以供调用
func_dict = func_input.to_dict()

In [17]:
func_dict.keys()

dict_keys(['title', 'start', 'end', 'function', 'n', 'nan', 'vc_base', 'index', 'columns'])

In [21]:
#执行程序部分：接受1.原始数据，2.break down维度拆分字典，3，function字典,输出报告所需的分析数据
#需要先在output文件夹下创建output.xlsx空表,也可以用python创建，但默认格式和字体美观性差
#注意：当output表非常长，超过1000行后，效率明显降低，原因可能是输出时为了在一张表上顺序排列，每输出一个问题，就会读取一次当前表的长度
#可以尝试分表输出来解决，这里放在一张表上
for i in range(func_input.shape[0]):
    print(str(i+1)+'/'+str(func_input.shape[0]),func_dict['title'][i])
    for k,v in input_dict.items():
        if func_dict['function'][i] == 'mean_arrange':
            locals()[func_dict['function'][i]](v,d,func_dict['columns'][i],k+'_'+func_dict['title'][i],func_dict['n'][i],func_dict['nan'][i])
        elif func_dict['function'][i] == 'sum_percentage':
            locals()[func_dict['function'][i]](v,d,func_dict['columns'][i],k+'_'+func_dict['title'][i],func_dict['n'][i])
        elif func_dict['function'][i] == 'vc':
            locals()[func_dict['function'][i]](v,d,func_dict['columns'][i],k+'_'+func_dict['title'][i],func_dict['vc_base'][i])
        else:
            locals()[func_dict['function'][i]](v,d,func_dict['columns'][i],k+'_'+func_dict['title'][i])

1/83 Question7
2/83 Question8
3/83 Question9
4/83 Question10
5/83 Question11
6/83 Question12
7/83 Question20
8/83 Question21
9/83 Question22
10/83 Question24
11/83 Question24
12/83 Question26
13/83 Question27
14/83 Question28
15/83 Question30
16/83 Question32
17/83 Question34
18/83 Question36
19/83 Question37
20/83 Question38
21/83 Question39
22/83 Question40
23/83 Question41
24/83 Question42
25/83 Question43
26/83 Question44
27/83 Question45
28/83 Question46
29/83 Question47
30/83 Question48
31/83 Question55
32/83 Question55
33/83 Question62
34/83 Question70
35/83 Question71
36/83 Question79
37/83 Question79
38/83 Question86
39/83 Question86
40/83 Question96
41/83 Question106
42/83 Question115
43/83 Question115
44/83 Question120
45/83 Question120
46/83 Question124
47/83 Question124
48/83 Question128
49/83 Question136
50/83 Question141
51/83 Question142
52/83 Question144
53/83 Question150
54/83 Question151
55/83 Question152
56/83 Question153
57/83 Question160
58/83 Question165
59/83 Qu

In [ ]:
#当程序出错时，根据中断的的索引debug
i = 15
for k,v in input_dict.items():
        if func_dict['function'][i] == 'mean_arrange':
            locals()[func_dict['function'][i]](v,d,func_dict['columns'][i],k+'_'+func_dict['title'][i],func_dict['n'][i],func_dict['nan'][i])
        elif func_dict['function'][i] == 'sum_percentage':
            locals()[func_dict['function'][i]](v,d,func_dict['columns'][i],k+'_'+func_dict['title'][i],func_dict['n'][i])
        elif func_dict['function'][i] == 'vc':
            locals()[func_dict['function'][i]](v,d,func_dict['columns'][i],k+'_'+func_dict['title'][i],func_dict['vc_base'][i])
        else:
            locals()[func_dict['function'][i]](v,d,func_dict['columns'][i],k+'_'+func_dict['title'][i])